In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import glob
import warnings


### Open historical and RCP8.5 hfds data, average over chosen periods, combine into dataset, and take the difference to get OHU change, save

In [ ]:

if cmip6:
    ens='cmip6'
    MODELS_do = [ 'ACCESS-ESM1-5', 'CESM2-WACCM','FGOALS-f3-L', 'GFDL-CM4',
                 'GFDL-ESM4', 'HadGEM3-GC31-LL',  'MPI-ESM1-2-HR',
                 'MPI-ESM1-2-LR',  'UKESM1-0-LL','HadGEM3-GC31-MM','CESM2','FIO-ESM-2-0','CNRM-CM6-1','CMCC-CM2-SR5']
else:
    ens='cmip5'
    MODELS_do = ['ACCESS1-0', 'ACCESS1-3', 'CMCC-CESM', 'CNRM-CM5',
                 'CSIRO-Mk3-6-0',  'GISS-E2-R-CC', 'MIROC-ESM-CHEM',
                 'MPI-ESM-LR', 'MRI-CGCM3', 'EC-EARTH', 'MIROC5','MIROC-ESM','GISS-E2-R','CMCC-CM'  ]


data_dir = '/gws/nopw/j04/ukfafmip/users/enewsom/OHUE_RCP85/'
files_hfds_hist =  glob.glob(data_dir+ 'hfds_*yr_*_historical*regridded*.nc') #this is a list of historical (or control) heat flux files
files_hfds_85 =  glob.glob(data_dir+ 'hfds_*yr_*_ssp585*regridded*.nc') #this is a list of perturbed heat flux files


period_c = slice(0,40) #however long of a period you want to average the controls; for the historical i used the first 40 years.
period_p = slice(66,87) #the years you want to average over in the perturbed runs, I used the final two decades of RCP8.5

HX0=[]
HX=[]

for count,model in enumerate(MODELS_do): 

    files_h0 = [match for match in files_hfds_hist if model in match] #picks out the file for that model (in your case, experiment)...
    files_h2 = [match for match in files_hfds_85 if model in match]
    
    file = files_h0[0]
    dc  = xr.open_dataset(file).assign_attrs(source_id=model).isel(time = period_c)
    Hc= dc['hfds'].mean('time')
    Hc=Hc.assign_coords(model=model)
    HX0.append(Hc) 
        
    file = files_h2[0]
    dc  = xr.open_dataset(file).assign_attrs(source_id=model).isel(time = period_p)
    Hc= dc['hfds'].mean('time')
    Hc=Hc.assign_coords(model=model)
    HX.append(Hc) 
    
    clear_output(wait=True)

H_control_model=xr.concat(HX0,dim='model',coords='minimal',compat='override') #this would instead concatenate a datarray of experiments...
H_co2_model=xr.concat(HX,dim='model',coords='minimal',compat='override') #same
H_anom = H_co2_model-H_control_model
H_anom.to_netcdf('OHU_change_'+ens+'.nc')

### Open historical and RCP8.5 SST and TAS data, average over chosen periods, combine into dataset, and take the difference to get SST or TAS change, save

In [11]:
files_tos_hist =  glob.glob(data_dir+ 'tos_*yr_*_historical*regridded*.nc') #list of files for control SSTs
files_tas_hist =  glob.glob(data_dir+ 'tas_yr_*_historical*regridded*.nc')  #list of files of perturbed SSTs
files_tos_85 =  glob.glob(data_dir+ 'tos_*yr_*_ssp585*regridded*.nc') #list of files for control surface temp (tas)
files_tas_85 =  glob.glob(data_dir+ 'tas_yr_*_ssp585*regridded*.nc') #list of files for perturbed surface temp (tas)


tas=0  #I didn't loop over these since you only have SST (I think)? so probably just take out tas stuff
tos=1
if tas:
    var='tas'
    varT='TAS'
else:
    var='tos'
    varT='SST'

TX=[]
TX0=[]

for count,model in enumerate(MODELS_do):
    print (model)
    
    if tas:
        files_h0 = [match for match in files_tas_hist if model in match] 
        files_h2 = [match for match in files_tas_85 if model in match]
    
    if tos:
        files_h0 = [match for match in files_tos_hist if model in match] 
        files_h2 = [match for match in files_tos_85 if model in match]
        
    clear_output(wait=True)

    file = files_h0[0]      
    dc  = xr.open_dataset(file).assign_attrs(source_id=model).isel(time = period_c)
    Tc= dc[var].mean('time')
    Tc=Tc.assign_coords(model=model)
    TX0.append(Tc) 

    file = files_h2[0]
    dc  = xr.open_dataset(file).assign_attrs(source_id=model).isel(time = period_p)
    Tc= dc[var].mean('time')
    Tc=Tc.assign_coords(model=model)
    TX.append(Tc) 

T_control_model=xr.concat(TX0,dim='model',coords='minimal',compat='override')
T_co2_model=xr.concat(TX,dim='model',coords='minimal',compat='override')
T_anom = T_co2_model-T_control_model
T_anom.to_netcdf(varT+'_change_'+ens+'.nc')

/home/users/enewsom/.conda/envs/nbenv2/lib/python3.10/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/users/enewsom/.conda/envs/nbenv2/lib/python3.10/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/users/enewsom/.conda/envs/nbenv2/lib/python3.10/site-packages/xarray/core/indexing.py:422: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=None)
